In [16]:
#!pip install google-cloud-pubsub

In [17]:
#copy the file from GCS to local
#!gsutil cp gs://cloud_storage_bucket/CA_Renewable_Prod_2010-2018.csv CA_Renewable_Prod_2010-2018.csv 

In [18]:
# create a topic
!gcloud pubsub topics create python-topic-test

Created topic [projects/friendly-plane-294914/topics/python-topic-test].


In [19]:
# library imports
import pandas as pd
from concurrent import futures
from google.cloud import pubsub_v1
from typing import Callable
import time
import json

In [20]:
# read the csv file
df = pd.read_csv('CA_Renewable_Prod_2010-2018.csv')

In [21]:
#size of the dataframe
df.shape

(67584, 10)

In [22]:
df.rename(columns={'SMALL HYDRO':'SMALL_HYDRO','SOLAR PV':'SOLAR_PV','SOLAR THERMAL':'SOLAR_THERMAL','WIND TOTAL':'WIND_TOTAL'}, inplace=True)


In [23]:
df.columns

Index(['TIMESTAMP', 'BIOGAS', 'BIOMASS', 'GEOTHERMAL', 'Hour', 'SMALL_HYDRO',
       'SOLAR', 'SOLAR_PV', 'SOLAR_THERMAL', 'WIND_TOTAL'],
      dtype='object')

In [24]:
# sample data
df.head(3)

,TIMESTAMP,BIOGAS,BIOMASS,GEOTHERMAL,Hour,SMALL_HYDRO,SOLAR,SOLAR_PV,SOLAR_THERMAL,WIND_TOTAL
0,2011-07-21 00:00:00,177.0,372.0,984.0,1.0,516.0,0.0,NaN,NaN,1607.0
1,2011-07-21 01:00:00,176.0,373.0,984.0,2.0,509.0,0.0,NaN,NaN,1619.0
2,2011-07-21 02:00:00,177.0,371.0,985.0,3.0,506.0,0.0,NaN,NaN,1660.0


In [25]:
json_records = df.to_json(orient='records')

In [26]:
messages = json.loads(json_records)

In [27]:
# project details
project_id = "your_project_id"
topic_id = "python-topic-test"

In [28]:
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_id)
publish_futures = []

In [29]:
for message in messages:
    message_json = json.dumps(message)
    message_bytes = message_json.encode('utf-8')
    publisher.publish(topic_path, data=message_bytes)
    time.sleep(10)

KeyboardInterrupt: 